<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/MIDO_MidiFile_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Extraction Work

Re-write of MIDO MIDI file work using python objects

# Setup notebook env

In [3]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# test ..
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root 

In [1]:
# install required libs
!pip install mido


     |████████████████████████████████| 61kB 3.1MB/s 


In [0]:

# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# imports
import pandas as pd
import mido 
from mido import MidiFile



# Python class to wrapper mido.MidiFile objects

Need to clean up code, make it re-usable for parsing multiple MIDI file performance files.



- time signature gathered from MIDI message within file, meta 'time_signature'
- typically 96 - 480 ticks per beat, from 'mido.MidiFile.ticks_per_beat' attribute
- tempo in microseconds per beat, gathered from 'set_tempo' MIDI message in file, converted using mido.tempo2bpm/ bpm2tempo, e.g. 434783 is 138 bpm
- conversion from ticks to seconds -> e.g. mido.tick2second(34, 480, 434783)
microseconds per tick = microseconds per quarter note / ticks per quarter note




In [0]:
class MIDI_File_Wrapper:
  ''' utility wrapper for loading, parsding a mido.MidiFile object'''

  def __init__(self, file_name):
    self.my_file_name = file_name
    self.my_file_midi = None    
    self.my_tempo = None  # stored as mido.Message instance
    self.my_time_sig = None # stored as mido.Message instance


    # load & configure file info...
    self.parse_file()

  # For call to str(). Prints readable form 
  def __str__(self): 
    return str('file: {}'.format(self.my_file_midi))
    
  
  def parse_file(self):
    ''' file must be: MIDI type 0 only;  one and only one tempo and time_sig meta messages in file. '''

    print('Loading file: {}'.format(self.my_file_name))

    # load file
    midi_file = MidiFile(self.my_file_name)
    self.my_file_midi = midi_file 

    # check it's the right type ..
    if midi_file.type != 0:
      raise ValueError('ERROR! Can only process type 0 files, this file is type: {}'.format(midi_file.type))


    # parse messages for time_sig and tempo info ..
    for msg in midi_file:

      if msg.type == 'time_signature':
        print('time sig: {}'.format(msg))

        # make sure no time sig changes
        if self.my_time_sig != None:
          raise ValueError('ERROR! more than one time sig: {}, {}'.format(self.my_time_sig, msg))
      
        self.my_time_sig = msg

      elif msg.type == 'set_tempo':

        print('tempo: {}'.format(msg))

        # make sure no tempo changes
        if self.my_tempo != None:
          raise ValueError('ERROR! more than one tempo: {}, {}'.format(self.my_tempo, msg))
        
        self.my_tempo = msg

    # now check we actually have tempo and time_sig set, or complain...
    if self.my_time_sig is None:
      raise ValueError('ERROR! no time signature found: {}'.format(midi_file))
    if self.my_tempo is None:
      raise ValueError('ERROR! no tempo found: {}'.format(midi_file))

    # load messages into DF

    # additional processing, e.g. cum_sum for clicks, milliseconds from start, etc


  def my_ticks(self):
    ''' Returns number of MIDI ticks configured in this file'''
    return self.my_file_midi.ticks_per_beat

  def my_length(self):
    ''' returns running time in seconds'''
    return self.my_file_midi.length




## Testing the above class...


In [118]:

file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'

mfw = MIDI_File_Wrapper(file_1)


Loading file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=434783 time=0>


## Test loading lots of files, creating lots of custom objects..



In [151]:
# testing parsing sub directory..

# reminder: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/

import os

for dirpath, dirs, files in os.walk('/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/'):
  for f in files:
    print('in subdir: {}, first file: {}'.format(dirpath, f)) 
    break

 

in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session2, first file: 4_jazz_120_beat_3-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session3, first file: 6_dance-disco_120_beat_4-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session, first file: 1_funk-groove1_138_beat_4-4.mid
in subdir: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/session1, first file: 123_funk_95_fill_4-4.mid


In [152]:
import glob

root_dir = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# recursively find all MIDI files..
my_files = []
for filename in glob.iglob(root_dir + '**/*.mid', recursive=True):
  my_files.append(filename)

# show what I found
my_files
print()
print('total # {}'.format(len(my_files)))


['/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/6_hiphop-groove6_87_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/8_rock-groove8_65_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/3_soul-groove3_86_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/9_soul-groove9_105_beat_4-4.mid',
 '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/5_funk-groove5_84_beat_4-4


total # 10


## Test creating lots of custom objects..

On creating a MIDI_File_Wrapper instance, a bunch of testing is performed, so the following is a good test to check a whole bunch of files at once to verify they meet pre-conditions...

In [143]:

my_midi_files = []

for f in my_files:
  my_midi_files.append(MIDI_File_Wrapper(f))

print('number objects create: {}'.format(len(my_midi_files)))

Loading file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=434783 time=0>
Loading file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=571429 time=0>
Loading file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid
time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
tempo: <meta message set_tempo tempo=588235 time=0>
Loading file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/4_soul-groove4_80